In [5]:
import requests
import json
from pprint import pprint

# Your configuration
url = "http://localhost:8089/api/v1/docsearch"
headers = {
    "api-key": "814dabdd0d755dbea52e46c4d3b01ee21d550650f8cb735b9b3489d5223bfa27"
}

payload = {
    "accountName": "FDRN_PEELHUNT",
    "searchText": "trading",
    "marketIds": "GB",
    "dateFrom": "2023-01-01",
    "dateTo": "2023-12-31",
    "searchTop": "1"
}

try:
    # Make request
    response = requests.post(url, headers=headers, json=payload)
    data = response.json()
    print(json.dumps(data, indent=2, ensure_ascii=False))
    
except requests.exceptions.RequestException as e:
    print(f"Error making request: {e}")
    
except json.JSONDecodeError as e:
    print(f"Error parsing JSON: {e}")
    
except Exception as e:
    print(f"Unexpected error: {e}")

{
  "header": {
    "recordCount": 1,
    "statusCode": 0,
    "statusMsg": ""
  },
  "documents": [
    {
      "docGuid": "a7a39aa8-ba82-41c2-b7bf-da414bcdf422",
      "docId": 9876,
      "docType": {
        "id": 300
      },
      "rixmlType": "Company Note Long",
      "markets": [
        {
          "id": "GB"
        }
      ],
      "sectors": [
        {
          "id": 8
        }
      ],
      "industries": [
        {
          "id": 14
        }
      ],
      "authors": [
        {
          "id": 21623
        }
      ],
      "corps": [
        {
          "id": 517
        }
      ],
      "docTitle": "#TRENDING",
      "docSynopsis": "·  LBG is one of the UK’s largest social digital publishers, focused on a notoriously difficult demographic to reach – youths.|·  We address near-term headwinds from declining ad yields with conservatism in our forecasts, and steps that management have taken.|·  Going forward, we see many growth avenues, including building up a US Di

In [14]:
import requests
import json
from pprint import pprint

# API endpoint
headers = {
    "X-Peelhunt-Token": "cda674de2e7aa9d79553660b1297e32a9bbf619b741020a896ee3d1700e5aa07"
}

url = 'https://api.efadrin.io/api/docsearch'

payload = {
    "AccountName": "FDRN_PEELHUNT",
    "searchText": "trading",
    # "marketIds": "GB",
    "authorIds": "21623",
    "dateFrom": "2023-01-01",
    "dateTo": "2023-12-31",
    "searchTop": "10"
}

try:
    # Make request
    response = requests.post(url, headers=headers, json=payload)
    data = response.json()
    print(json.dumps(data, indent=2, ensure_ascii=False))

except requests.exceptions.RequestException as e:
    print(f"Error making request: {e}")

except json.JSONDecodeError as e:
    print(f"Error parsing JSON: {e}")

except Exception as e:
    print(f"Unexpected error: {e}")

{
  "header": {
    "recordCount": 10,
    "statusCode": 0,
    "statusMsg": ""
  },
  "documents": [
    {
      "docGuid": "a7a39aa8-ba82-41c2-b7bf-da414bcdf422",
      "docId": 9876,
      "docType": {
        "id": 300
      },
      "rixmlType": "Company Note Long",
      "markets": [
        {
          "id": "GB"
        }
      ],
      "sectors": [
        {
          "id": 8
        }
      ],
      "industries": [
        {
          "id": 14
        }
      ],
      "authors": [
        {
          "id": 21623
        }
      ],
      "corps": [
        {
          "id": 517
        }
      ],
      "docTitle": "#TRENDING",
      "docSynopsis": "·  LBG is one of the UK’s largest social digital publishers, focused on a notoriously difficult demographic to reach – youths.|·  We address near-term headwinds from declining ad yields with conservatism in our forecasts, and steps that management have taken.|·  Going forward, we see many growth avenues, including building up a US D

In [18]:
import requests
import json
from pprint import pprint

# Your configuration
url = "https://api.efadrin.io/api/v1/docsearch"
headers = {
    "X-Peelhunt-Token": "cda674de2e7aa9d79553660b1297e32a9bbf619b741020a896ee3d1700e5aa07"
}

payload = {
    "accountName": "FDRN_PEELHUNT",
    "searchText": "trading",
    "marketIds": "GB",
    "dateFrom": "2023-01-01",
    "dateTo": "2023-12-31",
    "searchTop": "1"
}

try:
    # Make request
    response = requests.post(url, headers=headers, json=payload)
    data = response.json()
    print(json.dumps(data, indent=2, ensure_ascii=False))

except requests.exceptions.RequestException as e:
    print(f"Error making request: {e}")

except json.JSONDecodeError as e:
    print(f"Error parsing JSON: {e}")

except Exception as e:
    print(f"Unexpected error: {e}")

{
  "header": {
    "recordCount": 1,
    "statusCode": 0,
    "statusMsg": ""
  },
  "documents": [
    {
      "docGuid": "a7a39aa8-ba82-41c2-b7bf-da414bcdf422",
      "docId": 9876,
      "docType": {
        "id": 300
      },
      "rixmlType": "Company Note Long",
      "markets": [
        {
          "id": "GB"
        }
      ],
      "sectors": [
        {
          "id": 8
        }
      ],
      "industries": [
        {
          "id": 14
        }
      ],
      "authors": [
        {
          "id": 21623
        }
      ],
      "corps": [
        {
          "id": 517
        }
      ],
      "docTitle": "#TRENDING",
      "docSynopsis": "·  LBG is one of the UK’s largest social digital publishers, focused on a notoriously difficult demographic to reach – youths.|·  We address near-term headwinds from declining ad yields with conservatism in our forecasts, and steps that management have taken.|·  Going forward, we see many growth avenues, including building up a US Di